In [ ]:
titulo = "¿El xmen no binario?, "
titulo_texto = ""
nombre_descargado = "video29.mp4" 
seleccion_voz = 2

In [ ]:
#Librerias y rutas
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from googletrans import Translator
from TTS.api import TTS
import moviepy.editor as mp
import librosa
import numpy as np
import soundfile as sf
from moviepy.editor import VideoFileClip, AudioFileClip, TextClip, CompositeVideoClip
import moviepy.editor as mpe
import cv2
from PIL import Image
import whisper
from IPython.display import Audio, display
import pyrubberband
import os
import gc
from pydub import AudioSegment
import imageio
import random

nombre_audio = ""
nombre_gifiz = ""
nombre_gifcen = ""
nombre_gifder = ""

if seleccion_voz == 1 :
    nombre_audio = "goth.wav"
    nombre_gifiz = "//monos//bonitaiz.gif"
    nombre_gifcen = "//monos//bonitacen.gif"
    nombre_gifder = "//monos//bonitader.gif"
elif seleccion_voz == 2:
    nombre_audio = "audio_ing_resumen.wav"
    nombre_gifiz = "//monos//gatomostacho1.gif"
    nombre_gifcen = "//monos//gatomostacho2.gif"
    nombre_gifder = "//monos//gatomostacho3.gif"
elif seleccion_voz == 3:
    nombre_audio = "shado.wav"
    nombre_gifiz = "//monos//peloniz.gif"
    nombre_gifcen = "//monos//peloncen.gif"
    nombre_gifder = "//monos//pelonder.gif"
elif seleccion_voz == 4: 
    nombre_audio = "ukyo.wav"
    nombre_gifiz = "//monos//coleiz.gif"
    nombre_gifcen = "//monos//colecer.gif"
    nombre_gifder = "//monos//coleder.gif"
elif seleccion_voz == 5:
    nombre_audio = "jira.wav"
    nombre_gifiz = "//monos//rubiz.gif"
    nombre_gifcen = "//monos//rubcen.gif"
    nombre_gifder = "//monos//rubder.gif"
elif seleccion_voz == 6:
    nombre_audio = "bonita.wav"
    nombre_gifiz = "//monos//bonitaiz.gif"
    nombre_gifcen = "//monos//bonitacen.gif"
    nombre_gifder = "//monos//bonitader.gif"
elif seleccion_voz == 7:
    nombre_audio = "boni.wav"
    nombre_gifiz = "//monos//roryiz.gif"
    nombre_gifcen = "//monos//rorycen.gif"
    nombre_gifder = "//monos//roryder.gif"

display(seleccion_voz)
display(nombre_audio)


rollitas = ["beach","docerola","final","funkcarioca","hotlanda","huasteco","intro",
            "mariachi","oaxaca","parade","shake","skyline","takes","wood"]

ruta_carpeta_principal = "C://Users//R20//Desktop//videostraduccidos//"
ruta_carpeta_audios_esp = "audios_creados//audio_esp_resumen.wav"
ruta_carpeta_audios_ing = "audios_creados//" + nombre_audio
ruta_carpeta_audios_fin = "audios_creados//audio_fin_resumen.wav"
ruta_carpeta_con_musica = "audios_creados//audio_fin_musica.wav"
temporales = "audios_creados//temporales//"
musica = "audios_creados//musica//"
ruta_carpeta_videofin = "videos_creados//" + nombre_descargado
ruta_carpeta_videofin_editado = "videos_creados//finales" + nombre_descargado
nombre_video = "videos_descargados//" + nombre_descargado

rutaVideo = ruta_carpeta_principal + nombre_video
ruta_musica =  ruta_carpeta_principal + musica
ruta_video_final_editado = ruta_carpeta_principal + ruta_carpeta_videofin_editado
rutaAudioIngles = ruta_carpeta_principal + ruta_carpeta_audios_ing
rutaAudioGenerado =  ruta_carpeta_principal + ruta_carpeta_audios_esp
ruta_audios_temporales = ruta_carpeta_principal + temporales
rutaAudioGeneradoNuevaVelocidad = ruta_carpeta_principal + ruta_carpeta_audios_fin
rutaAudioGeneradoMusicaFondo = ruta_carpeta_principal +ruta_carpeta_con_musica
ruta_video_final = ruta_carpeta_principal + ruta_carpeta_videofin
ruta_imagenes = ruta_carpeta_principal + "imagenes//"
ruta_audio = "C://Users//R20//Desktop//videostraduccidos//audios_creados//audio_con_voz.wav"
gc.collect()

In [ ]:
#poner musica de fondo

random_number = np.random.randint(0, 14)
# Cargar los audios
audio = AudioSegment.from_wav(ruta_audio)
music = AudioSegment.from_wav(ruta_musica + rollitas[random_number] + ".wav")

max_amplitude = audio.max_dBFS
max_music = music.max_dBFS
display(max_amplitude)
display(max_music)
display(str(random_number) + " - " + str(rollitas[random_number]))

# Adjust the volume of the music (optional)
if(max_amplitude <= -1):
    audio_scaled = audio - .50
    display("holi")
music_scaled = music - 25

# Overlay the audio and music
audio_mixed = audio.overlay(music_scaled)

# Save the mixed audio
audio_mixed.export(rutaAudioGeneradoMusicaFondo, format="wav")

audio_widget = Audio(filename=rutaAudioGeneradoMusicaFondo, autoplay=False)
display(audio_widget)

In [7]:
#videos individuales, largo, gif y subtitulado.
numero_audios = 0
video = None 

gifs_clips = []
for filename in os.listdir(ruta_audios_temporales):
     if filename.endswith(("wav")):
         numero_audios +=1
gc.collect()


def animacion_lateral(image,iteraciones_for,n_segundos,n_frames,duration_original):
    # Estirar la imagen (ejemplo: duplicar su ancho)
        image = image.resize((image.width , image.height ))

        # Convertir a formato compatible con OpenCV
        image_opencv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGRA)

        # Crear la animación
        alto, ancho, _ = image_opencv.shape
        if iteraciones_for % 2 == 0:
            matriz_transformacion = np.array([[2, 0, -image.width], [0, 1, 1]], dtype=np.float32)
        else:
            matriz_transformacion = np.array([[2, 0, 1], [0, 1, 1]], dtype=np.float32)
        frames = []
        pixeles_por_segundo = image.width / n_segundos
        pixeles_por_frame = pixeles_por_segundo/n_frames
        
        for i in range(int(n_segundos*n_frames)):
            imagen_transformada = cv2.warpAffine(image_opencv, matriz_transformacion, (ancho, alto))
            # Convertir de BGR a RGB para mantener los colores originales
            imagen_transformada = cv2.cvtColor(imagen_transformada, cv2.COLOR_BGRA2RGB)
            frames.append(imagen_transformada)
            if iteraciones_for % 2 == 0:
                matriz_transformacion[0, 2] += pixeles_por_frame
            else:
                matriz_transformacion[0, 2] -= pixeles_por_frame

        # Crear el clip de video con la animación
        videok = mpe.ImageSequenceClip(frames, fps=30).set_duration(duration_original / 1.5)
        return videok
def animacion_zoom(duration_original,fr,imagen,iteraciones_for):
    image = imageio.imread(imagen)
    image_opencv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGRA)
    n_frames = int(duration_original * fr)
    alto, ancho, _ = image.shape
    frames = []
    # Calcular la escala inicial y final para que la animación dure todo el tiempo del video
    if iteraciones_for % 2 == 0:
        escala_inicial = 1
        escala_final = 2
    else:
        escala_inicial = 2
        escala_final = 1

    for i in range(n_frames):
        # Calcular la escala actual en función del progreso del video
        progreso = i / n_frames
        escala_actual = escala_inicial + (escala_final - escala_inicial) * progreso
        
        # Aplicar la transformación de escala
        matriz_transformacion = cv2.getRotationMatrix2D((ancho/2, alto/2), 0, escala_actual)
        imagen_transformada = cv2.warpAffine(image, matriz_transformacion, (ancho, alto))
        
        # Crear el borde negro para mantener el tamaño del video
        borde_negro = np.zeros((imagen_transformada.shape[0], 10, 4), dtype=np.uint8)
        imagen_con_borde = np.concatenate((imagen_transformada, borde_negro), axis=1)
        frames.append(imagen_con_borde)

    # Crear el clip de video con la secuencia de imágenes y establecer la duración
    clip = mpe.ImageSequenceClip(frames, fps=fr).set_duration(duration_original / 1.5)
    return clip

def generar_videos_individuales():
    display(numero_audios)
    iteraciones_for = 2
    n_frames = 30
    random_anterior = 0
    textos_separados= ["","","Los mejores noodles"]
    for y in range(numero_audios):
        ruta_audio_for = ruta_audios_temporales + "temporal" + str(y) + ".wav"
        ruta_imagen_for = ruta_imagenes + str(y+1)+ ".png"
        y_generated, sr_generated = librosa.load(ruta_audio_for)
        duration_original = len(y_generated) / sr_generated
        n_segundos = duration_original
        iteraciones_for +=1
        # Cargar la imagen
        image = Image.open(ruta_imagen_for)

        numero_random = random.randint(1, 4)
        while numero_random == random_anterior:
            numero_random = random.randint(1, 4)
        random_anterior = numero_random
        if numero_random == 1 or numero_random == 2:
            video = animacion_lateral(image,numero_random,n_segundos,n_frames,duration_original)
        else:
            video = animacion_zoom(duration_original,n_frames,ruta_imagen_for,numero_random)
        #video = clip.set_audio(mpe.AudioFileClip(ruta_audio_for))

        video.fps = 30  # Ajusta el frame rate

        video = video.resize((1920, 1080))  # Dimensiones Full HD
        video_path = ruta_carpeta_principal + "//videos_creados//finales//temporal" + str(y) + ".mp4"
        if (((y + 1) % 3 == 0 or y == 0 ) and y != numero_audios) and y < len(textos_separados):
            texto_completo = textos_separados[y]  # Texto que deseas mostrar
            mitad = len(texto_completo) // 2
            indice_espacio_mas_cercano = texto_completo.rfind(" ", 0, mitad)  # Busca el espacio más cercano a la mitad
            if indice_espacio_mas_cercano == -1:  # Si no se encontró ningún espacio, usar la mitad como punto de división
                indice_espacio_mas_cercano = mitad
            primera_parte = texto_completo[:indice_espacio_mas_cercano]
            segunda_parte = texto_completo[indice_espacio_mas_cercano:].strip()
            texto_con_salto_de_linea = f"{primera_parte}\n{segunda_parte}"
            fontsize = 65
            posicion = ("center")
            subtitle = mpe.TextClip(texto_con_salto_de_linea, fontsize=fontsize, font="ProtestStrike-Regular.ttf", color='white', bg_color='black')
            subtitle_clip = subtitle.set_pos(posicion).set_duration(duration_original / 1.5)
            videos = mpe.CompositeVideoClip([video, subtitle_clip])
            ffmpeg_params = ["-vcodec", "h264_nvenc"]  # Utiliza la GPU Nvidia para la codificación de video
            videos.write_videofile(video_path, codec='libx264', ffmpeg_params=ffmpeg_params)
            #videos.write_videofile(video_path)
        else:
            ffmpeg_params = ["-vcodec", "h264_nvenc"]  # Utiliza la GPU Nvidia para la codificación de video
            video.write_videofile(video_path, codec='libx264', ffmpeg_params=ffmpeg_params)
        
            #video.write_videofile(video_path)
        
        del y_generated, sr_generated, duration_original, image, video
        gc.collect()  # Llamar al recolector de basura de Python para liberar memoria adicional

def video_largo():
    video_concatenado = None
    for y in range(numero_audios):
        if(y != 0):
            video_path_nuevo = ruta_carpeta_principal + "//videos_creados//finales//temporal" + str(y) + ".mp4"
            video_nuevo = mpe.VideoFileClip(video_path_nuevo)

            # Concatenar los dos videos
            if y == 1:
                video_path_anterior = ruta_carpeta_principal + "//videos_creados//finales//temporal" + str(y-1) + ".mp4"
                video_anterior = mpe.VideoFileClip(video_path_anterior)
                video_concatenado = mpe.concatenate_videoclips([video_anterior, video_nuevo])
            else:
                video_concatenado = mpe.concatenate_videoclips([video_concatenado, video_nuevo])
        gc.collect()
    # Agregar el audio al video concatenado
    video_concatenado_with_audio = video_concatenado.set_audio(mpe.AudioFileClip(rutaAudioGeneradoMusicaFondo))

    # Guardar el video concatenado con audio
    video_concatenado_path = ruta_carpeta_principal + "//videos_creados//finales//video_final.mp4"
    ffmpeg_params = ["-vcodec", "h264_nvenc"]  # Utiliza la GPU Nvidia para la codificación de video
    video_concatenado_with_audio.write_videofile(video_concatenado_path, codec='libx264', ffmpeg_params=ffmpeg_params)
    #video_concatenado_with_audio.write_videofile(video_concatenado_path)
    # Liberar recursos
    gc.collect()

def poner_gif():
        #Generar letreros en el video.
    
    #imagen = (mpe.ImageClip(ruta_imagenes + "conocidas//patito.gif")
    imagen = (mpe.VideoFileClip(ruta_imagenes + "conocidas//"+nombre_gifiz, has_mask=True)         
            .resize((800, 600))
            .set_duration(8)
                )  # Establecer la duración de la imagen a 3 segundos
    imagen_final = (mpe.VideoFileClip(ruta_imagenes + "conocidas//"+nombre_gifcen, has_mask=True)         
            .resize((800, 600))
            .set_duration(8)
                )
    imagen_medio = (mpe.VideoFileClip(ruta_imagenes + "conocidas//"+nombre_gifder, has_mask=True)         
            .resize((800, 600))
            .set_duration(8)
                )

    random_number = np.random.randint(1, 5)
    if random_number == 1:
        clip_inicio = imagen.set_pos("center")#((-250, 750))
        clip_final = imagen_final.set_start(video.duration - 30).set_pos("center")
        clip_medio = imagen_medio.set_start(video.duration//2 - 1).set_pos("center")
    elif random_number == 2:
        clip_inicio = imagen.set_pos("center")
        clip_final = imagen_medio.set_start(video.duration - 30).set_pos("center")
        clip_medio = imagen_final.set_start(video.duration//2 - 1).set_pos("center")
    elif random_number == 3:
        clip_inicio = imagen_medio.set_pos("center")
        clip_final = imagen.set_start(video.duration - 30).set_pos("center")
        clip_medio = imagen_final.set_start(video.duration//2 - 1).set_pos("center")
    elif random_number == 4:
        clip_inicio = imagen_final.set_pos("center")
        clip_final = imagen.set_start(video.duration - 30).set_pos("center")
        clip_medio = imagen_medio.set_start(video.duration//2 - 1).set_pos("center")



    superposicion = clip_inicio
    superposicion2 = clip_final
    superposicion3 = clip_medio


    gifs_clips.append(superposicion)
    gifs_clips.append(superposicion2)
    gifs_clips.append(superposicion3)
    display("Termine de poner los gifs")
    gc.collect()

def poner_imagenes(nombre_imagen, inicio):
    #Cargar la imagen y redimensionarla
    imagen = (mpe.ImageClip(ruta_imagenes + "conocidas//"+nombre_imagen+".png")
            .resize((400, 300))
            .set_duration(1)
            .set_start(inicio)
            )  # Establecer la duración de la imagen a 3 segundos

    ancho_video, alto_video = video.size

    # Calcular las coordenadas del centro
    x_centro = ancho_video / 2 - imagen.w / 2
    y_centro = alto_video / 2 - imagen.h / 2

    superposicion = imagen.set_pos((x_centro, y_centro))
    return superposicion

def obtener_texto_voz():
    model = whisper.load_model("medium")
    result = model.transcribe(ruta_audio)

    whisper_text = result["text"]
    whisper_language = result['language']

    print("Termine de generar el texto del audio")
    whisper_text = whisper_text.replace('.', '. ').replace('?', '? ')
    return  whisper_text

def poner_subtitulos():
    palabras_clave = ["STEAM","XBOX","NINTENDO","MARVEL","UCM","DC","DCEU","NETFLIX","NICK"]  # Ajusta según tus necesidades
    whisper_text = obtener_texto_voz()

    words = whisper_text.split()

    # Group the words into pairs
    #subtitle_lines = [' '.join(words[i:i+2]) for i in range(0, len(words), 2)]
    subtitle_lines = []
    i = 0
    while i < len(words):
        if i + 1 < len(words) and words[i+1].strip() != '':
            # Si el siguiente elemento de la lista no está vacío, unir las dos palabras con un espacio
            subtitle_lines.append(' '.join([words[i], words[i+1]]))
            i += 2
        else:
            # Si el siguiente elemento de la lista está vacío o es el último elemento, agregar solo la palabra actual
            subtitle_lines.append(words[i])
            i += 1

    # Remove any empty strings from the list of subtitle lines
    subtitle_lines = [line.strip() for line in subtitle_lines if line.strip()]

    # Create a list to store the text clips
    subtitle_clips = []
    duration = video.duration / len(subtitle_lines)

    # Initialize a variable to keep track of the current line number
    line_number = 0
    
    # Create a TextClip for each subtitle line and add it to the list
    for i, line in enumerate(subtitle_lines):
        if any(palabra in line for palabra in palabras_clave):
            # Encontrar la primera palabra clave en la línea
            for palabra in palabras_clave:
                if palabra in line:
                    palabra_clave_encontrada = palabra
                    break
            display(palabra_clave_encontrada)      
            superposicion = poner_imagenes(palabra_clave_encontrada, i * duration)
            gifs_clips.append(superposicion)

        # Increment the line number
        line_number += 1
    # Overlay the subtitle clips on the video clip
    video_con_superposicion = CompositeVideoClip([video]+ gifs_clips)
    video_con_superposicion = video_con_superposicion.set_duration(video.duration)
    ffmpeg_params = ["-vcodec", "h264_nvenc"]  # Utiliza la GPU Nvidia para la codificación de video
    video_con_superposicion.write_videofile(ruta_video_final_editado, codec='libx264', ffmpeg_params=ffmpeg_params)
    #video_con_superposicion.write_videofile(ruta_video_final_editado)
    gc.collect()

def poner_intro():
        video_path_nuevo = ruta_carpeta_principal + "//videos_creados//intromorobos.mp4"
        video_nuevo = mpe.VideoFileClip(video_path_nuevo)
        video_path_anterior = ruta_video_final_editado
        video_anterior = mpe.VideoFileClip(video_path_anterior)
        outro = ruta_carpeta_principal + "//videos_creados//intromorobos.mp4"
        video_outro = mpe.VideoFileClip(outro)
        video_concatenado = mpe.concatenate_videoclips([video_nuevo, video_anterior])
        video_concatenad2 = mpe.concatenate_videoclips([video_concatenado, video_outro])
        video_concatenado_path = ruta_carpeta_principal + "//videos_creados//finales//video_finaloso.mp4"
        ffmpeg_params = ["-vcodec", "h264_nvenc"]  # Utiliza la GPU Nvidia para la codificación de video
        video_concatenad2.write_videofile(video_concatenado_path, codec='libx264', ffmpeg_params=ffmpeg_params)
        gc.collect()


#generar_videos_individuales()
#video_largo()
video = mp.VideoFileClip(ruta_carpeta_principal + "//videos_creados//finales//video_final.mp4")
poner_gif()
poner_subtitulos()
poner_intro()

'Termine de poner los gifs'

Termine de generar el texto del audio
Moviepy - Building video C://Users//R20//Desktop//videostraduccidos//videos_creados//finalesvideo29.mp4.
MoviePy - Writing audio in finalesvideo29TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video C://Users//R20//Desktop//videostraduccidos//videos_creados//finalesvideo29.mp4



Moviepy - Done !
Moviepy - video ready C://Users//R20//Desktop//videostraduccidos//videos_creados//finalesvideo29.mp4
Moviepy - Building video C://Users//R20//Desktop//videostraduccidos////videos_creados//finales//video_finaloso.mp4.
MoviePy - Writing audio in video_finalosoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video C://Users//R20//Desktop//videostraduccidos////videos_creados//finales//video_finaloso.mp4



Moviepy - Done !
Moviepy - video ready C://Users//R20//Desktop//videostraduccidos////videos_creados//finales//video_finaloso.mp4
